# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1) # generování dat 

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons # jediný co musíme je specifikovat dimensionalitu dat
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation   # 10 nodů a aktivace relu
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation   # další layer sigmoid - cca funkce jako goniometrická
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [10]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [11]:
!pip install pydot


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# TODO: To make the following line work you need to install graphviz (if you have not done so in one of the previous classes)
# 1) follow the instructions https://graphviz.gitlab.io/download/?fbclid=IwAR1V-lrRhho5rSfBVYXYISsighqRwOCOgMHLmL_DclkQrPtMXQaKj3mFcqs
# 2) this notebook has been tested with version 8.0.3
# 3) make sure you add it to the PATH variable (you are specifically asked during the installation) at least for local user

tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Model compilation and training

In [13]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers   # to je to theta co máš na papíře  - 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [14]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.5252023 ]
 [ 0.4082964 ]
 [-0.3925295 ]
 [-0.05197519]
 [-0.27463418]
 [ 0.58512396]
 [ 0.11228704]
 [-0.66524297]
 [-0.31026563]
 [-0.43334484]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[ 0.36062902 -0.5500564  -0.5148962  -0.4063329   0.0262484   0.09897244
  -0.30671236  0.12054098  0.46010852  0.43828106]
 [-0.5130485   0.00500596  0.0661124  -0.24158278 -0.284894   -0.17337394
   0.09588879  0.4449073   0.08536756 -0.4214123 ]
 [ 0.24379277  0.48436666  0.13884085 -0.015558    0.2876184   0.5599636
   0.02685225 -0.06317604 -0.5419383  -0.4293604 ]
 [-0.22925434  0.456447   -0.01109952  0.14400029  0.29951334 -0.2750419
   0.11873007  0.47647655  0.0654912  -0.37117517]
 [-0.24255186  0.48977518  0.46724546  0.5064784  -0.39718613  0.48474658
  -0.5117652   0.01748419  0.15786111 -0.12553456]
 [ 0.4461187  -0.5444481  -0.5230943   0.10250556  0.39682573  

In [32]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 928us/step - auc_1: 0.6480 - binary_accuracy: 0.6109 - loss: 0.6761
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - auc_1: 0.8853 - binary_accuracy: 0.8082 - loss: 0.4652
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - auc_1: 0.9125 - binary_accuracy: 0.8372 - loss: 0.4013
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 851us/step - auc_1: 0.9261 - binary_accuracy: 0.8546 - loss: 0.3597
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - auc_1: 0.9337 - binary_accuracy: 0.8635 - loss: 0.3361
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - auc_1: 0.9395 - binary_accuracy: 0.8675 - loss: 0.3199
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9415 - binary_accuracy: 0.8722 - loss: 0.3121
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9483 - binary_accuracy: 0.8814 - loss: 0.2941
Epoch 9/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - auc_1: 0.9475 - binary_accuracy: 0.8816 - loss: 0.2

In [35]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - auc_1: 0.9759 - binary_accuracy: 0.9309 - loss: 0.2007
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step


In [36]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.60
0 - 0.23
0 - 0.18
1 - 1.00
0 - 0.00
0 - 0.03
1 - 0.98
0 - 0.00
1 - 0.92
1 - 1.00


### Add early stopping and regularization

první věc která se dá přidat. 
Nejčastejší problém proč nepoužijeme neuronovou síť je že overfituje

In [38]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs) # l2 regularizátor
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

ValueError: Cannot convert '8' to a shape.

In [39]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - auc_1: 0.9792 - binary_accuracy: 0.9333 - loss: 0.1866 - val_auc_1: 0.9788 - val_binary_accuracy: 0.9322 - val_loss: 0.1898
Epoch 2/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9784 - binary_accuracy: 0.9334 - loss: 0.1894 - val_auc_1: 0.9788 - val_binary_accuracy: 0.9334 - val_loss: 0.1877
Epoch 3/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9801 - binary_accuracy: 0.9331 - loss: 0.1829 - val_auc_1: 0.9792 - val_binary_accuracy: 0.9353 - val_loss: 0.1873
Epoch 4/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9800 - binary_accuracy: 0.9358 - loss: 0.1845 - val_auc_1: 0.9793 - val_binary_accuracy: 0.9322 - val_loss: 0.1880
Epoch 5/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9803 - binary_accuracy: 0.9356 - loss: 0.1835 - val_auc_1: 0.9794 - val_binary_accuracy: 0.9331 - val_loss: 0.1863
Epoch 6/200
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc_1: 0.9829 - binary_accuracy: 0.9396 - loss: 0.

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [40]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 10
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

ValueError: Cannot convert '8' to a shape.

In [ ]:
%load_est tensorboard
%tensorboard --logdir=logs/binary_clssification_test